PA 1
* For task 2.1 and 3.1, find YOUR CODE HERE and implement.
* For task 2.2 and 3.2, find YOUR ANSWER HERE and write your answer.

In [1]:
# ## Set up Jupyter Kernel, select Python 3 (ipykernel).
# ## Only need to run once.
# ## restart the kernel after finishing installation

# !pip install "ray==2.10.0"
# !pip install "pyarrow==18.1.0"
# !pip install "modin==0.37.0"

In [2]:
## Verify installations:
!python -c "import ray; print('ray:', ray.__version__)"
!python -c "import modin; print('modin:', modin.__version__)"

ray: 2.10.0
modin: 0.37.0


## Task 2
#### Task 2.1

In [19]:
# Note: We include only 10M data in the csv file to fit the memory of Datahub.

import ray, json, os
import time
import numpy as np
import modin.pandas as pd
from modin.utils import reload_modin
ray.shutdown()
reload_modin()
ray.init(num_cpus=4)

def run_task2(path):
    ## DO NOT MODIFY: START 
    start_time = time.perf_counter()
    raw_df = pd.read_csv(path)
    ## DO NOT MODIFY: End 

    # YOUR CODE HERE
    # Compact vote parsing: normalize missing, convert comma-decimals, remove thousands commas,
    # coerce to numeric, fill missing with 0, and clip extreme outliers at the 99th percentile.
    v = raw_df['vote'].replace('-', pd.NA).astype(str).str.strip()
    v = v.str.replace(r',(?=\d{1,2}$)', '.', regex=True).str.replace(',', '', regex=False)
    v = v.replace({'nan': None, '': None})
    # Convert to numeric and keep NaN for missing values so mean() ignores them
    raw_df['vote'] = pd.to_numeric(v, errors='coerce')
    raw_df['rating_bucket'] = raw_df['overall'].round().astype('Int64')
    # Convert unixReviewTime to year
    raw_df['reviewYear'] = pd.to_datetime(raw_df['unixReviewTime'], unit='s').dt.year

    # Treat missing votes as 0 for the avg_helpful_votes_per_review calculation
    raw_df['vote_filled'] = raw_df['vote'].fillna(0.0)
    output = raw_df.groupby('rating_bucket').agg(
    	n_reviews=('reviewerID', 'size'),
    	n_unique_reviewers=('reviewerID', 'nunique'),
    	sum_helpful_votes=('vote_filled', 'sum'),
    	earliest_review_year=('reviewYear', 'min')
    ).reset_index()
    # compute average helpful votes per review using sum / n_reviews (includes missing as zeros)
    output['avg_helpful_votes_per_review'] = output['sum_helpful_votes'] / output['n_reviews']
    output = output.drop(columns=['sum_helpful_votes'])
    
    ## DO NOT MODIFY: START 
    submit = output.describe().round(2)
    duration_s = time.perf_counter() - start_time
    print(f"Processing time (excluding file write): {duration_s:.3f} s")
    current_data = json.loads(submit._to_pandas().to_json())
    return current_data
    ## DO NOT MODIFY: END 

2025-10-11 22:02:02,214	INFO worker.py:1752 -- Started a local Ray instance.


In [20]:
def compare_json_files(current_data):
        # Load both JSON files
    path = os.path.expanduser("~/public/origin_results_PA1_task2_1.json")
    with open(path, 'r') as f:
        origin_data = json.load(f)
        
    # Track if any error > 1% exists
    has_error_over_1_percent = False
    
    # Compare each metric
    for metric in origin_data.keys():
        print(f"\n=== Comparing {metric} ===")
        
        for stat in origin_data[metric].keys():
            origin_val = origin_data[metric][stat]
            current_val = current_data[metric][stat]
            
            # Calculate error percentage
            if origin_val != 0:
                error_percent = abs(current_val - origin_val) / abs(origin_val) * 100
            else:
                error_percent = 0 if current_val == 0 else float('inf')
            
            # Only print if error > 1%
            if error_percent > 1.0:
                print(f"  {stat}: {current_val} vs {origin_val} (error: {error_percent:.2f}%)")
                has_error_over_1_percent = True
    
    # Assert if there exists an error > 1%
    assert not has_error_over_1_percent, "Found errors greater than 1% between the JSON files"
    
    if not has_error_over_1_percent:
        print("\n✓ All comparisons within 1% tolerance")

In [21]:
raw_dataset_path = "~/public/modin_dev_dataset_10M_rows.csv"
current_data = run_task2(raw_dataset_path)
compare_json_files(current_data)

Processing time (excluding file write): 20.736 s

=== Comparing rating_bucket ===

=== Comparing n_reviews ===

=== Comparing n_unique_reviewers ===

=== Comparing avg_helpful_votes_per_review ===

=== Comparing earliest_review_year ===

✓ All comparisons within 1% tolerance


#### Task 2.2
Change the number of CPUs used by Modin or the Ray backend ([documentation here](https://modin.readthedocs.io/en/stable/getting_started/using_modin/using_modin_locally.html#advanced-configuring-the-resources-modin-uses)) on your instance and run your data manipulation code. Document the execution times you see with 1, 2, 3, and 4 CPUs. Is it a linear speedup? If not, why?

Note:
* We won't check the processing time for Task2. So in your submission, it doesn't matter how many CPUs you set in ray.init(num_cpus=X).

In [22]:
def measure_cpu_scaling(path, cpus=(1, 2, 3, 4)):
    """Measure run_task2 runtime with different Ray CPU counts.

    This re-initializes Ray/Modin for each CPU count, runs the same pipeline, and
    records wall-clock time. It prints per-CPU times and speedups relative to 1 CPU.
    """
    timings = {}
    for c in cpus:
        print(f"\n--- Measuring with {c} CPU(s) ---")
        try:
            ray.shutdown()
        except Exception:
            pass
        # reload Modin to pick up the new Ray init
        try:
            reload_modin()
        except Exception:
            pass
        ray.init(num_cpus=c)

        t0 = time.perf_counter()
        # run_task2 prints its own processing time; we measure end-to-end here
        _ = run_task2(path)
        t1 = time.perf_counter()
        elapsed = t1 - t0
        timings[c] = elapsed
        print(f"Total elapsed (including re-init): {elapsed:.3f} s")

    # summarize
    base = timings.get(cpus[0], None)
    print("\nCPU scaling summary:")
    for c in cpus:
        e = timings[c]
        if base and base > 0:
            speedup = base / e
        else:
            speedup = float('nan')
        print(f" CPUs={c}: {e:.3f}s  speedup={speedup:.2f}x")

    return timings

# If run interactively, run the measurement for 1-4 CPUs and print results.
try:
    _ = measure_cpu_scaling(raw_dataset_path, cpus=(1, 2, 3, 4))
except Exception:
    # measurement is optional; if it fails in your environment, ignore and continue
    pass


--- Measuring with 1 CPU(s) ---


2025-10-11 22:06:29,846	INFO worker.py:1752 -- Started a local Ray instance.


Processing time (excluding file write): 59.802 s
Total elapsed (including re-init): 59.810 s

--- Measuring with 2 CPU(s) ---


2025-10-11 22:07:35,629	INFO worker.py:1752 -- Started a local Ray instance.


Processing time (excluding file write): 33.104 s
Total elapsed (including re-init): 33.110 s

--- Measuring with 3 CPU(s) ---


2025-10-11 22:08:14,693	INFO worker.py:1752 -- Started a local Ray instance.


Processing time (excluding file write): 25.291 s
Total elapsed (including re-init): 25.298 s

--- Measuring with 4 CPU(s) ---


2025-10-11 22:08:46,121	INFO worker.py:1752 -- Started a local Ray instance.


Processing time (excluding file write): 20.944 s
Total elapsed (including re-init): 20.948 s

CPU scaling summary:
 CPUs=1: 59.810s  speedup=1.00x
 CPUs=2: 33.110s  speedup=1.81x
 CPUs=3: 25.298s  speedup=2.36x
 CPUs=4: 20.948s  speedup=2.86x


The speedup is not linear (ideal would be 4× at 4 CPUs). Using Amdahl’s-law style reasoning, the observed speedups imply ≈86–90% of the workload is parallelizable; the remaining serial fraction, plus scheduling/serialization and I/O overheads, limit scaling. 

In practice the following factors cause the sub-linear speedup:
1. Non-parallel work (I/O, CSV parsing, small serial steps) — Amdahl’s law limits maximum speedup.
2. Ray/Modin overheads (task scheduling, serialization/deserialization, actor startup).
3. Data skew and partition imbalance (some partitions/partitions take longer → stragglers).
4. Resource contention (disk bandwidth, memory pressure, GC) when adding workers.

Practical takeaway: expect sub-linear but meaningful speedups; to improve scaling, profile the pipeline, minimize serialization and small tasks, tune partitioning, and avoid re-initializing Ray in the timed loop.

## Task 3
#### Task 3.1

In [23]:
"""
Plain merge sort algorithm 
"""
import heapq
from typing import List
import time
import ray
import numpy as np
import json

num_workers = 4
ray.shutdown()
ray.init(num_cpus=num_workers)

def merge(sublists: List[list]) -> list:
    """
    Merge sorted sublists into a single sorted list.

    :param sublists: List of sorted lists
    :return: Merged result
    """
    result = []
    sublists = [sublist for sublist in sublists if len(sublist)> 0]
    heap = [(sublist[0], i, 0) for i, sublist in enumerate(sublists)]
    heapq.heapify(heap)
    while len(heap):
        val, i, list_ind = heapq.heappop(heap)
        result.append(val)
        if list_ind+1 < len(sublists[i]):
            heapq.heappush(heap, (sublists[i][list_ind+1], i, list_ind+1))
    return result

def plain_merge_sort(collection: list, npartitions: int = 4) -> list:
    """
    Sorts a list using the merge sort algorithm. Breaks the list into multiple partitions.

    :param collection: A mutable ordered collection with comparable items.
    :return: The same collection ordered in ascending order.

    Time Complexity: O(n log n)

    Examples:
    >>> merge_sort([0, 5, 3, 2, 2])
    [0, 2, 2, 3, 5]
    >>> merge_sort([])
    []
    >>> merge_sort([-2, -5, -45])
    [-45, -5, -2]

    Modified from: https://github.com/TheAlgorithms/Python/
    """

    if len(collection) < npartitions:
        return sorted(collection)
    breaks = [i*len(collection)//npartitions for i in range(npartitions)]
    breaks.append(len(collection))
    sublists = [collection[breaks[i]:breaks[i+1]] for i in range(len(breaks)-1)]
    sorted_sublists = [plain_merge_sort(sublist, npartitions=2) for sublist in sublists] # just use 2 partitions in recursive calls
    return merge(sorted_sublists)

@ray.remote
def ray_merge_sort(collection, pair, npartitions):
    return plain_merge_sort(collection[pair[0]:pair[1]], npartitions)

2025-10-11 22:21:56,105	INFO worker.py:1752 -- Started a local Ray instance.


In [24]:
def merge_sort_ray(collection_ref: ray.ObjectRef, length: int, npartitions: int = 4) -> list:
    """
    Merge sort with ray
    """
    ## DO NOT MODIFY: START    
    breaks = [i*length//npartitions for i in range(npartitions)]
    breaks.append(length)
    # Keep track of partition end points
    sublist_end_points = [(breaks[i], breaks[i+1]) for i in range(len(breaks)-1)]
    ## DO NOT MODIFY: END
    
    # YOUR CODE HERE
    # Launch Ray tasks for each partition
    futures = [
        ray_merge_sort.remote(collection_ref, pair, npartitions)
        for pair in sublist_end_points
    ]
    sorted_sublists = ray.get(futures)
    # Pass your list of sorted sublists to `merge`
    return merge(sorted_sublists)

In [25]:
# We will be testing your code for a list of size 10M. Feel free to edit this for debugging. 
list1 = list(np.random.randint(low=0, high=1000, size=2_000_000))
list2 = [c for c in list1] # make a copy
length = len(list2)
list2_ref = ray.put(list2) # insert into the driver's object store

start1 = time.time()
list1 = plain_merge_sort(list1, npartitions=num_workers)
end1 = time.time()
time_baseline = end1 - start1
print("Plain sorting:", time_baseline)

start2 = time.time()
list2 = merge_sort_ray(collection_ref=list2_ref, length=length, npartitions=num_workers)
end2 = time.time()
time_ray = end2 - start2
print("Ray sorting:", time_ray)

speedup = time_baseline / time_ray
print("Speedup: ", speedup)
# Save timing metrics to JSON
results = {
    "time_baseline": time_baseline,
    "time_ray": time_ray,
    "speedup": speedup
}
assert sorted(list1) == list2, "Sorted lists are not equal"
assert speedup > 1.4

Plain sorting: 15.175149202346802
Ray sorting: 9.113381385803223
Speedup:  1.6651502400620004


#### Task 3.2
The ideal speedup you can get for a task with 4 workers is, of course, 4. Can you estimate the theoretical maximum speedup you can get for the above merge sort algorithm (in terms of the time for sorting sublists, and the time for merging)? How do you account for the difference between the theoretical result and the observed speedup with Ray?

1. Theoretical maximum speedup for this algorithm:

- If T_sort is the (parallelizable) time to sort sublists and T_merge is the (serial) time to merge them, serial time = T_sort + T_merge.
With perfect parallelism (p workers), ideal time ≈ T_sort/p + T_merge. So maximum speedup as p→∞ is:
- S_max = (T_sort + T_merge)/T_merge = 1 + T_sort/T_merge = 1 + P/(1−P).
- P ≈ 0.53 gives S_max ≈ 1 + 0.53/0.47 ≈ 2.14, so even with infinite parallel sorting capacity, the algorithm (as implemented) cannot exceed ≈2.14× speedup because merging is a substantial fraction (~47%) of the serial time.

2. Accounting for the difference to the observed speedup:
- The merge phase T_merge is serial in our implementation (final heap-merge on the driver), so it limits speedup by Amdahl’s law. Even with perfect parallel sorting, we cannot exceed 1 + T_sort/T_merge.
- Practical overheads (scheduling, serialization, data movement, object copies, driver-side merge work, load imbalance, GC and memory pressure, I/O) further reduce observed speedup below the ideal S(p).